In [ ]:
import numpy as np
import IPython
from IPython.display import Audio
import torch
import torch.nn as nn
import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# from data import AudioData, get_dataloaders
from model_training.data import AudioData, get_dataloaders

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def display(data, fs=16000):
    IPython.display.display(Audio(data, rate=fs))

In [ ]:
ad = AudioData(n_mfcc = 40)

In [ ]:
train_dataloader, test_dataloader = get_dataloaders(ad)

In [ ]:
for e in train_dataloader:
    X,y = e
    decoded_X = ad.decode(X)
    display(decoded_X)
    X = X[None, :]
    print(X.shape)
#     print(y.shape)
#     decoded_y = ad.decode(y)
#     display(decoded_y)
    break

In [ ]:
class IDK(nn.Module):
    def __init__(self, n_mfcc=40):
        super(IDK, self).__init__()
        self.linear = torch.nn.Linear(n_mfcc,n_mfcc)
        
    def forward(self, x):
        x = self.linear(x)
        return x

In [ ]:
linear = IDK(n_mfcc=40).to(device)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn_layers = torch.nn.Sequential(
            torch.nn.Conv2d(1, 8, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.Conv2d(8, 16, kernel_size=3),
            torch.nn.ReLU()
        )
        
        self.linear_layers = torch.nn.Linear(16 * 24 * 24, 10)
        
    def forward(self, x):
        x = self.cnn_layers(x)
        x = torch.flatten(x, 1)
        x = self.linear_layers(x)
        return x

In [ ]:
cnn = CNN().to(device)
print(cnn)

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear.parameters())

EPOCHS = 1

# cnn_losses = []
# cnn_validation_loss = []

linear_losses = []
linear_validation_loss = []

for i in range(EPOCHS):
    for batch in tqdm.tqdm(train_dataloader):
#         cnn.zero_grad()
        linear.zero_grad()
        batchX, batchY = batch
        batchX = torch.unsqueeze(batchX, 0)

        batchX = batchX.type(torch.FloatTensor)
        batchY = batchY.type(torch.FloatTensor)

        batchX = batchX.to(device)
        batchY = batchY.to(device)
        
        batchX = torch.squeeze(batchX)
        batchY = torch.squeeze(batchY)
        

#         out = cnn(batchX)
        out = linear(batchX)
        l = loss(out, batchY)
        l.backward()
#         cnn_losses.append(l)
        linear_losses.append(l)
        optimizer.step()

#     with torch.no_grad():
# #         out = cnn(valX)
#         out = linear(valX)
#         L = loss(out, valY)
# #         cnn_validation_loss.append(float(L))
#         linear_validation_loss.append(float(L))

In [ ]:
plt.plot(linear_losses)

In [ ]:
for validation in train_dataloader:
    valX, valY = validation
    
    valX = valX.type(torch.FloatTensor)
    valY = valY.type(torch.FloatTensor)

    valX = torch.squeeze(valX).to(device)
    valY = torch.squeeze(valY).to(device)
    
    out = linear(valX).cpu()
    out = torch.unsqueeze(out, 0)
    out = ad.decode(out.detach().numpy())
    out = display(out)
    
    out1 = out.cpu().detach().numpy()
    
    
    plt.plot(out1)
    break